In [ ]:
from ipyleaflet import Map, TileLayer, FullScreenControl, ScaleControl

m = Map(center=(47, 2), zoom=5)

radon = TileLayer(url='https://cartoradon.irsn.fr/static/r/{z}/{x}/{y}.png',
                  tms=True, opacity=0.5)
m.add_layer(radon)

fullscreen = FullScreenControl()
m.add_control(fullscreen)
scalecontrol = ScaleControl(position='bottomleft')
m.add_control(scalecontrol)
m

